Import JSON files and merge them to one pandaframe 


In [1]:
import numpy as np
import json
import pandas as pd
import os
import datetime

In [2]:
path = 'C:/Users/Maison/Documents/INFO-H423/Projet Data Mining/JSON/'
path_gtfs ='C:/Users/Maison/Documents/INFO-H423/Projet Data Mining/GTFS3sep/gtfs3sept.zip'

In [3]:
# creating a list conatining all my json files
list_files=[]
for i in os.listdir(path):
    if i.endswith('.json'):
        list_files.append(path + i)

#print(list_files)

In [4]:
#merge the data into one pandaframe which contains every record of timestamp, line id, direction id, point id and the distance from the last point 
listvehiculepositions=[]
for file in list_files:#je parcours ma liste cntenant mes json files
    #print('Uploading the file' + ':' +file )
    with open(file) as f : 
        data = json.load(f) #je load les json files
        f.close()
        for d in data['data']:    
            for line in d['Responses']:  
                if line is not None : # la 10ième fois qu'on appelle l'API il y a juste le temps et rien dans l'élément lines donc on vérifie que ce ne soit pas pas vide
                    for position in line['lines']:  
                        for i in position['vehiclePositions']:   
                            d_time = d['time']
                            position_lineId = position['lineId']
                            i_directionId = i['directionId']
                            i_distancefromPoint = i['distanceFromPoint']
                            i_pointId = i['pointId']
                            listvehiculepositions.append([d_time,position_lineId,i_directionId,i_distancefromPoint,i_pointId]) #construction de la liste des listes
print('Uploading of the files is finished')
df_vehiculepositions = pd.DataFrame(data=listvehiculepositions, columns = ['time','lineId', 'directionId','distanceFromPoint','pointId'])# on va en faire un panda frame 

KeyboardInterrupt: 

In [5]:
print(df_vehiculepositions.lineId.value_counts())

95    531447
7     529144
82    515924
81    504996
51    500110
       ...  
33     69363
76     63121
72     42247
69     33083
77     19044
Name: lineId, Length: 74, dtype: int64


In [5]:
df_vehiculepositions.time = pd.to_numeric(df_vehiculepositions.time, downcast = "float")

##Division par 1000 pour passer de ms à s /!\ Attention à le faire qu'une fois
df_vehiculepositions.time = df_vehiculepositions.time/1000 

In [6]:
import datetime
#df_vehiculepositions['time'].sort_values()
def datefromtimestamp(timestamp):
    date = datetime.datetime.fromtimestamp(timestamp).strftime('%m-%d-%Y %H:%M:%S')
    return date
time_fun = datefromtimestamp(df_vehiculepositions.time[0])
print(time_fun)
my_date = datetime.datetime(2021,9,3)
print(my_date)
my_stamp = datetime.datetime.timestamp(my_date)
print(my_stamp)


09-06-2021 09:53:36
2021-09-03 00:00:00
1630620000.0


C:\Users\Maison\AppData\Local\Temp\ipykernel_1904\2640931946.py:4: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  date = datetime.datetime.fromtimestamp(timestamp).strftime('%m-%d-%Y %H:%M:%S')


In [8]:
#import the merged pandaframe into a csv file 
df_vehiculepositions.to_csv(path + 'dataentier.csv')


Import gtfs line

In [5]:
import gtfs_functions as gtfs
import matplotlib
import geopandas
import matplotlib.pyplot as plt

In [6]:
routes, stops, stop_times, trips, shapes = gtfs.import_gtfs(path_gtfs,busiest_date=False)

Maintenant il faut pouvoir allez sélectionner à partir d'un numéro de ligne (lineId) d'un stop (stop_id) et d'un jour l'horaire correspondant dans le GTFS, tout en faisant attention aux exceptions

In [215]:
calendardf = pd.read_table('calendar.txt',sep=',')          #export the calendar file into data frame
calendarDatesdf = pd.read_table('calendar_dates.txt',sep=',')  #also export the exceptions

In [223]:
def FromDayToService(date):
    '''Determine all the service_id concerned by a date, given in the format date=('20210907','tuesday')
        it first take a look at every regular service_id then examine the exceptions. 
        Returns a list of service_id of type string'''

    ListServiceId=[]
    for i in range(len(calendardf)):
        startdate,endate=calendardf.iloc[i,[8,9]]
        if int(date[0]) in range(int(startdate),int(endate)+1):
            if calendardf.loc[i,date[1]]==1:
                #It appends every service_id which contains the date, and for which there is a '1'
                # under right day in the week
                ListServiceId.append(str(calendardf.iloc[i,0]))     

    #Now we have to take a look a the exceptions
    #We selections all the exceptions in calendarDatesdf that concerns the date in the input
    ModifServiceId= calendarDatesdf[calendarDatesdf['date']==int(date[0])][['service_id','exception_type']]
    print('Number of service_id involved in modifications: '+ str(len(ModifServiceId)))
    #for every modif we check the kind of exception
    for modif in ModifServiceId.iterrows():  
        service = str(modif[1]['service_id'])
        exception = modif[1]['exception_type']
        if exception == 1:
            #if the exception is 1, it means that we should change to a 1 the cell related to the date in
            # input, in the row equal to service in calendar
            ListServiceId.append(service)
            #print('ServiceId: '+str(service)+' correclty added' )
        elif (exception == 2) & (service in ListServiceId):
            #if the exception is 2, it means that we should change to a 0 the cell related to the date in
            # input, in the row equal to service in calendar
            #Since this service_id must have been selected in the first loop (in Calendar), we need to remove it
            ListServiceId.remove(service)
            #print('ServiceId: '+str(service)+' correclty removed' )
        elif (exception == 2) & (service not in ListServiceId):
            #if the exception is 2 but we haven't found the service id in the first loop, this is weird
            #In principle this should not happend, but we're never too carefull
            print('Warning : exception 2 but no service_id = ' +service+ ' found in calendar')
    return (ListServiceId)

def FromLineIdtoRouteId(LineId):
    
    '''Takes a lineid in argument a returns the correspondent routeid, since stop_times only 
        contains route_id'''

    routeId = routes[routes['route_short_name']==LineId]['route_id'].iloc[0]
    return routeId


def ScheduledTime(date,lineId,stopId):

    '''Takes a triplet (date,lineId,stopId) with date in the format date=('20210907','tuesday')
        and returns the schedule for that triplet. The schedule is composed as follows: if we are in a period of
        punctuality then returns the (arrivaltime,true), and if we are in regularity, 
        returns (TimeOfWaitFromPreviousVehicle,False), all in one list '''

    ListOfService = pd.Series(FromDayToService(date))  #List of service_id tha apply for that day
    RouteId = FromLineIdtoRouteId(lineId)               #We need the routeid to seek in the GTFS
    #We select the relevant rows in GTFS stop_times
    Scheduledf = stop_times[(stop_times['stop_id']==stopId)&(stop_times['route_id']==RouteId)&(stop_times['service_id'].isin(ListOfService))].sort_values('arrival_time')
    ArrivalTime = Scheduledf['arrival_time'].values
    #print('Number of service_id involved: '+ str(len(ListOfService)))
    #We compute every wainting time to determine punctuality of regularity (< or > 720 s)
    #Schedule = list(ArrivalTime[1:]-ArrivalTime[:-1])
    Schedule = list(ArrivalTime[1:]-ArrivalTime[:-1])
    Schedule.insert(0,ArrivalTime[0])
    #True if punctuality, False if regularity
    Punctuality = np.array(Schedule)>720 
    #Ultimately, if we are a punctuality, we want the arrival time and if we are in regularity, we want the 
    #time bewteen two vehicles
    for i,punctuality in enumerate(Punctuality):
        if punctuality:
            if (0<i<(len(Punctuality)-1)):
                if ~(Punctuality[i-1]) & ~(Punctuality[i+1]):
                    Punctuality[i]=False
            else:
                Schedule[i]=ArrivalTime[i]
 
    #Schedule = list(zip(Schedule,Punctuality))
    #The format is a list of tuples

    return Schedule,Punctuality

def RouteIdToLineId(routeId):
    lineId = routes[routes['route_id']==routeId]['route_short_name'].iloc[0]
    return lineId
        
def TroncatedScheduleRegularity(date,lineId,stopId):
    Schedulemom = ScheduledTime(date,lineId,stopId)
    TroncatedSchedule = [intervaltime for i,intervaltime in enumerate(Schedulemom[0]) if not(Schedulemom[1][i])]
    return TroncatedSchedule


In [190]:
StopTimesGroupTrip = stop_times.groupby('trip_id')[['stop_sequence']].mean().round()
StopTimesGroupTrip['stop_sequence']=StopTimesGroupTrip['stop_sequence'].astype(int)

test = pd.merge(stop_times, StopTimesGroupTrip, how='right', on= ['trip_id','stop_sequence'])

DicOfMiddleStop={}
test2 = test[['route_id','stop_sequence','stop_id','direction_id']]
for stop in test2['stop_id'].unique():
    test3 = test2[test2['stop_id']==stop]
    routeid = test3['route_id'].unique()[0]
    directionid = test3['direction_id'].unique()
    for direction in directionid:
        key = str(routeid)+':'+str(direction)
        if DicOfMiddleStop.get(key) is None:
            DicOfMiddleStop[key]=stop
DicOfWeek = {'weekday':('20210915','wednesday'),'saturday':('20210918','saturday'),'sunday':('20210919','sunday')}  
def DataForCluster(lineid, directionid,dayoftheweek):
    routeid = FromLineIdtoRouteId(lineid) 
    key = str(routeid)+':'+str(directionid)
    if DicOfMiddleStop.get(key)!=None:
        meanstop = DicOfMiddleStop[key]
        dataCluster = TroncatedScheduleRegularity(DicOfWeek[dayoftheweek],lineid,meanstop)
    return dataCluster


A titre de comparaison on peut aussi essayer avec le code de Mehdi, cad de travailler avec une version modifiée (complète) de calendar

In [43]:
calendardf = pd.read_table('calendar.txt',sep=',')          
calendarDatesdf = pd.read_table('calendar_dates.txt',sep=',')  

def date_to_day_of_week(date_str):
  # Parse the date string into a datetime object
  date = datetime.datetime.strptime(date_str, '%Y%m%d')

  # Return the day of the week (Monday, Tuesday, etc.)
  return date.strftime('%A').lower()
  
def update_calendar(calendar_df, calendar_dates_df):
    calendar_dates_df1 = calendar_dates_df[calendar_dates_df['exception_type'] == 1]
    calendar_dates_df2 = calendar_dates_df[calendar_dates_df['exception_type'] == 2]
    
    calendar_df.set_index('service_id', inplace = True)

    for index, elem in calendar_dates_df1.iterrows():
        date1 = str(calendar_dates_df1['date'][index])
        weekday = date_to_day_of_week(date1)
        service2change = calendar_dates_df1['service_id'][index]
        calendar_df.loc[service2change, weekday] = 1
        
    for index, elem in calendar_dates_df2.iterrows():
        date = str(calendar_dates_df2['date'][index])
        weekday = date_to_day_of_week(date)
        service2change = calendar_dates_df2['service_id'][index]
        calendar_df.loc[service2change, weekday] = 0

    return calendar_df

calendardf_update = update_calendar(calendardf, calendarDatesdf)


Ducoup on peut adapter légèrement les fonctions précédentes

In [42]:
def FromDayToServiceMehdi(date):
    '''Determine all the service_id concerned by a date, given in the format date=('20210907','tuesday')
        it first take a look at every regular service_id then examine the exceptions. 
        Returns a list of service_id of type string'''

    ListServiceId=[]
    for i in range(len(calendardf_update)):
        startdate,endate=calendardf_update.iloc[i,[7,8]]
        if int(date[0]) in range(int(startdate),int(endate)+1):
            if calendardf_update.loc[i,date[1]]==1:
                #It appends every service_id which contains the date, and for which there is a '1'
                # under right day in the week
                ListServiceId.append(str(calendardf_update.iloc[i,0]))     
    return ListServiceId


def ScheduledTimeMehdi(date,lineId,stopId):

    '''Takes a triplet (date,lineId,stopId) with date in the format date=('20210907','tuesday')
        and returns the schedule for that triplet. The schedule is composed as follows: if we are in a period of
        punctuality then returns the (arrivaltime,true), and if we are in regularity, 
        returns (TimeOfWaitFromPreviousVehicle,False), all in one list '''

    ListOfService = pd.Series(FromDayToServiceMehdi(date))  #List of service_id tha apply for that day
    RouteId = FromLineIdtoRouteId(lineId)               #We need the routeid to seek in the GTFS
    #We select the relevant rows in GTFS stop_times
    Scheduledf = stop_times[(stop_times['stop_id']==stopId)&(stop_times['route_id']==RouteId)&(stop_times['service_id'].isin(ListOfService))].sort_values('arrival_time')
    ArrivalTime = Scheduledf['arrival_time'].values
    print('Number of service_id involved: '+ str(len(ListOfService)))
    #We compute every wainting time to determine punctuality of regularity (< or > 720 s)
    Schedule = list(ArrivalTime[1:]-ArrivalTime[:-1])
    Schedule.insert(0,ArrivalTime[0])
    #True if punctuality, False if regularity
    Punctuality = np.array(Schedule)>720 
    #Ultimately, if we are a punctuality, we want the arrival time and if we are in regularity, we want the 
    #time bewteen two vehicles
    for i,punctuality in enumerate(Punctuality):
        if punctuality:
            Schedule[i]=ArrivalTime[i]
    Schedule = list(zip(Schedule,Punctuality))
    #The format is a list of tuples

    return Schedule

In [11]:
array = df_vehiculepositions.index.to_numpy()
date_init = datetime.datetime(2021, 9, 6, 4)
epoch_date_init = datetime.datetime.timestamp(date_init)
date_end = datetime.datetime(2021, 9, 7, 3) #L'idée c'est de commencer à 4h du mat et finir à 3h du matin le lendemain pour inclure les transports après minuit
epoch_date_end = datetime.datetime.timestamp(date_end)
print(epoch_date_init)
print(df_vehiculepositions.time[0])

    
    

1630893600.0
1630914800.0


In [12]:
df_dict = df_vehiculepositions.to_dict()

In [18]:
df_dict.keys()

dict_keys(['time', 'lineId', 'directionId', 'distanceFromPoint', 'pointId'])

In [13]:
import datetime
from tqdm import tqdm

def actual_time(line, stop, year, month, day):
    
    date_init = datetime.datetime(year, month, day, 4)
    epoch_date_init = datetime.datetime.timestamp(date_init)
    date_end = datetime.datetime(year, month, day+1, 3) #L'idée c'est de commencer à 4h du mat et finir à 3h du matin le lendemain pour inclure les transports après minuit
    epoch_date_end = datetime.datetime.timestamp(date_end)
    date_midnight = datetime.datetime(year, month, day, 0) #Obtain seconds after midnight format
    midnight_time = datetime.datetime.timestamp(date_midnight)
    listvehiculearrivals = []
    humanlistvehiculearrivals = []
    for index in tqdm(df_dict['time']):
        if epoch_date_init < df_dict['time'][index]:
            if epoch_date_end > df_dict['time'][index]:
                if df_dict['lineId'][index] == line:
                    if df_dict['pointId'][index] == stop:
                        if df_dict['distanceFromPoint'][index] == 0:
                            sec_after_midnight = df_dict['time'][index] -midnight_time
                            human_arrival_time = datefromtimestamp(df_dict['time'][index])
                            listvehiculearrivals.append(sec_after_midnight)
                            humanlistvehiculearrivals.append(human_arrival_time)
    return listvehiculearrivals, humanlistvehiculearrivals






    

Prenons maintenant un exemple pour que tout le monde voit bien, cherchons le RealTime et le ScheduleTime, sur la ligne du 4 à l'arrêt Héros

In [222]:
Date = ('20210915','wednesday')
lineid = '44'
stopid = '5561F'

Schedule,TronCated=ScheduledTime(Date,lineid,stopid),TroncatedScheduleRegularity(Date,lineid,stopid)

Number of service_id involved in modifications: 26
[ True  True  True  True  True  True False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True  True  True  True  True  True  True  True  True  True
  True  True]
Number of service_id involved in modifications: 26
[ True  True  True  True  True  True False False False False False False
 False F

In [189]:
Schedule[0]

[21000.0,
 22560.0,
 23640.0,
 24540.0,
 25320.0,
 600.0,
 600.0,
 720.0,
 780.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 780.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 540.0,
 180.0,
 720.0,
 780.0,
 720.0,
 180.0,
 540.0,
 540.0,
 180.0,
 720.0,
 540.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 480.0,
 420.0,
 480.0,
 480.0,
 420.0,
 480.0,
 420.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 480.0,
 420.0,
 420.0,
 420.0,
 480.0,
 480.0,
 420.0,
 480.0,
 420.0,
 720.0,
 660.0,
 660.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 81720.0,
 82620.0,
 83520.0,
 84420.0,
 85320.0,
 86220.0,
 87120.0,
 88020.0,
 88920.0,
 300.0,
 600.0,
 240.0]

In [188]:
TronCated

[600.0,
 600.0,
 720.0,
 780.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 780.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 540.0,
 180.0,
 720.0,
 780.0,
 720.0,
 180.0,
 540.0,
 540.0,
 180.0,
 720.0,
 540.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 480.0,
 420.0,
 480.0,
 480.0,
 420.0,
 480.0,
 420.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 420.0,
 480.0,
 480.0,
 420.0,
 420.0,
 420.0,
 480.0,
 480.0,
 420.0,
 480.0,
 420.0,
 720.0,
 660.0,
 660.0,
 720.0,
 720.0,
 720.0,
 720.0,
 720.0,
 300.0,
 600.0,
 240.0]

In [20]:
def ExcessWaitingTime()

{0: 1630914816.0,
 1: 1630914816.0,
 2: 1630914816.0,
 3: 1630914816.0,
 4: 1630914816.0,
 5: 1630914816.0,
 6: 1630914816.0,
 7: 1630914816.0,
 8: 1630914816.0,
 9: 1630914816.0,
 10: 1630914816.0,
 11: 1630914816.0,
 12: 1630914816.0,
 13: 1630914816.0,
 14: 1630914816.0,
 15: 1630914816.0,
 16: 1630914816.0,
 17: 1630914816.0,
 18: 1630914816.0,
 19: 1630914816.0,
 20: 1630914816.0,
 21: 1630914816.0,
 22: 1630914816.0,
 23: 1630914816.0,
 24: 1630914816.0,
 25: 1630914816.0,
 26: 1630914816.0,
 27: 1630914816.0,
 28: 1630914816.0,
 29: 1630914816.0,
 30: 1630914816.0,
 31: 1630914816.0,
 32: 1630914816.0,
 33: 1630914816.0,
 34: 1630914816.0,
 35: 1630914816.0,
 36: 1630914816.0,
 37: 1630914816.0,
 38: 1630914816.0,
 39: 1630914816.0,
 40: 1630914816.0,
 41: 1630914816.0,
 42: 1630914816.0,
 43: 1630914816.0,
 44: 1630914816.0,
 45: 1630914816.0,
 46: 1630914816.0,
 47: 1630914816.0,
 48: 1630914816.0,
 49: 1630914816.0,
 50: 1630914816.0,
 51: 1630914816.0,
 52: 1630914816.0,
 53